In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as pl

In [3]:
import scipy.optimize 

In [2]:
def NPV_enter(r,sigma,c,n,FC_mean,S):
    
    return (1+r)/r*(S/8*((1+sigma*n*c)/(1+sigma*n)-c)**2-FC_mean)

In [3]:
def NPV_exit(r,sigma,c,n,FC_mean,S,FC_i):
    
    return (1+r)/r*(S/8*((1+sigma*n*c)/(1+sigma*n)-c)**2)-(1+r)/(1+r-alpha)*FC_i-(1/r-alpha/(1+r-alpha))*FC_mean

In [4]:
def storyteller(alpha,rlist,sigma,c,n,Slist,epoch,lambd,Elist,CF):
    
    #scale - mean FC
    #r_initial=r
    
    FC_mean = lambd
    
    FC = np.random.exponential(lambd, size=n)
    
    FC_mean_story = [FC.mean()]
    n_story = [n]
    FC_max_story = [FC.max()]
    n_ex = [0]
    n_en = [0]
    
    for i in range(epoch):
        r=rlist.iloc[i]
        E=Elist.iloc[i]
        S=Slist.iloc[i]
        #r = rlist[i]
        #if i==restep:
            #r = r2
        #if i>restep and oneshock==1:
            #r=r_initial
        
        print(f"эпоха номер: {i}")
        
        en = 0
        
        while NPV_enter(r,sigma,c,len(FC)*CF,FC_mean,S)>E:
            
            FC = np.append(FC,np.random.exponential(lambd))
            en+=1
            
        n_en.append(en*CF)
            
        print(f"вход закончился, число фирм: {len(FC)*CF}, E: {E}, NPV: {NPV_enter(r,sigma,c,len(FC)*CF,FC_mean,S)}")
        
        for i in range(len(FC)):
            
            if np.random.uniform(0,1)>alpha:
                
                FC[i] =  np.random.exponential(lambd)
                
        FC = np.flip(np.sort(FC))
        
        print(f"пересчет закончился")
        

        i=0
        k=0
        
        while i < len(FC):
            
            if NPV_exit(r,sigma,c,len(FC)*CF,FC_mean,S,FC[i])<0:
                
                FC = np.delete(FC, i)
                k+=1
            else:   
                i+=1
        
        n_ex.append(k*CF)
                
        print(f"выход закончился, число фирм: {len(FC)*CF},NPV_exit {NPV_exit(r,sigma,c,len(FC)*CF,FC_mean,S,FC.max())}")
                
        
        FC_mean_story.append(FC.mean())
        FC_max_story.append(FC.max())
        n_story.append(len(FC)*CF)
    
    stats=pd.DataFrame(data=np.array([FC_mean_story,FC_max_story,n_story,n_ex,n_en]).T, 
                        columns=["FC_mean","FC_max","n","n_ex","n_en"])    
    stats["r"] = rlist
    stats["E"] = Elist
    stats["S"] = Slist
    return stats


In [16]:
sigma = 0.5
c = 0.5
n = 16
S = 1
r = 0.03


alpha = 0.5
lambd=0.005#это средние фиксде косты входящего
CF = 0.001
E = 0.005

epoch = 100
#r = np.random.normal(loc=0.06, scale=0.01, size=epoch)


data = pd.DataFrame()
#data["r"] = np.random.normal(0.03,0.01,epoch+1)
#data["E"] = np.random.normal(0.005,0.001,epoch+1)
data["r"] = [r]*epoch
data["E"] = [E]*epoch
S=[S]
gr=1.01
for i in range(epoch-1):
    S.append(S[-1]*gr)
data["S"] = S

In [31]:
scenario=storyteller(alpha,data["r"],sigma,c,n,data["S"],epoch,lambd,data["E"],CF)

эпоха номер: 0
вход закончился, число фирм: 2.9290000000000003, E: 0.005, NPV: 0.004981179561156858
пересчет закончился
выход закончился, число фирм: 2.7640000000000002,NPV_exit 0.00017512850721931872
эпоха номер: 1
вход закончился, число фирм: 2.954, E: 0.005, NPV: 0.00495149182192223
пересчет закончился
выход закончился, число фирм: 2.816,NPV_exit 0.00012814213148973352
эпоха номер: 2
вход закончился, число фирм: 2.979, E: 0.005, NPV: 0.0049308035706182324
пересчет закончился
выход закончился, число фирм: 2.845,NPV_exit 9.437630516961892e-05
эпоха номер: 3
вход закончился, число фирм: 3.003, E: 0.005, NPV: 0.0049896205257723165
пересчет закончился
выход закончился, число фирм: 2.865,NPV_exit 0.00011257422105093173
эпоха номер: 4
вход закончился, число фирм: 3.028, E: 0.005, NPV: 0.004986301964237649
пересчет закончился
выход закончился, число фирм: 2.896,NPV_exit 0.00013975060192431
эпоха номер: 5
вход закончился, число фирм: 3.053, E: 0.005, NPV: 0.004991718172456735
пересчет законч

In [32]:
def px(f, lambd=lambd):
    return lambd * np.exp(-lambd * f)

def Fx(f, lambd=lambd):
    return 1 - np.exp(-lambd * f)

K = scenario["n"].mean()
M = scenario["n_ex"].mean()
F = scenario["FC_max"].mean()
print(K, M, F)

print(
    (1+r)/r *( S[0]/8 * ((1-c)/(1+sigma*(M+K)))**2 - lambd) - E,
    (1+r)/r * S[0]/8 *((1-c)/(1+sigma*M))**2 - (1/r - alpha/(1+r-alpha)) * lambd - (1+r)/(1+r-alpha) * F,
    (1-alpha)*(1-Fx(F)) * M - K * Fx(F)
)

4.308188118811882 0.17773267326732678 0.012841013551248934
-0.07464728153710971 0.7180290187652287 0.08858403251719804


In [22]:
fig = pl.line({"fact_max" : scenario["FC_max"][10:]})
fig.show()

In [23]:
fig = pl.line({"fact" : scenario["n"][10:]})
fig.show()

In [24]:
fig = pl.line({"fact" : scenario["n_en"][10:]})
fig.show()

In [25]:
%run my_estimators2.ipynb

In [26]:
def estimator(x):
    
    a,l,c,s = x
    global scenario
    
    #scenario1=scenario[10:].copy()
    scenario1=scenario[2:].copy()
    
    P = 1-np.exp(-l*scenario1["FC_max"])
    scenario1["g1"] = (1+scenario1["r"])/scenario1["r"]*(scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"].shift()+scenario1["n_en"])))**2-1/l)-scenario1["E"]
    scenario1["g2"] = (1+scenario1["r"])/scenario1["r"]*scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"])))**2-(1/scenario1["r"]-a/(1+scenario1["r"]-a))*1/l-(1+scenario1["r"])/(1+scenario1["r"]-a)*scenario1["FC_max"]
    scenario1["g3"] = scenario1["n"]-scenario1["n"].shift()+(1-a)*(1-P)*scenario1["n"].shift()-P*scenario1["n_en"]
    scenario1["g4"] = -scenario1["FC_max"]*np.exp(-l*scenario1["FC_max"])-np.exp(-l*scenario1["FC_max"])/l+1/l-P*scenario1["FC_mean"]
    
    scenario["g1"]=scenario1["g1"]
    scenario["g2"]=scenario1["g2"]
    scenario["g3"]=scenario1["g3"]
    scenario["g4"]=scenario1["g4"]
    
    return scenario1["g1"].mean()**2+scenario1["g2"].mean()**2+scenario1["g3"].mean()**2+scenario1["g4"].mean()**2
    #return scenario["g1"].mean()**2,scenario["g2"].mean()**2,scenario["g3"].mean()**2,scenario["g4"].mean()**2

In [27]:
estimator((alpha,1/lambd,c,sigma))

6.7731425139734695e-06

In [28]:
scenario

,FC_mean,FC_max,n,n_ex,n_en,r,E,S,g1,g2,g3,g4
0,0.004584,0.029938,16.000,0.000,0.000,0.03,0.005,1.000000,NaN,NaN,NaN,NaN
1,0.004095,0.014181,2.758,0.171,2.913,0.03,0.005,1.010000,NaN,NaN,NaN,NaN
2,0.003979,0.012943,2.813,0.141,0.196,0.03,0.005,1.020100,NaN,0.001886,NaN,-0.000028
3,0.003894,0.012627,2.844,0.135,0.166,0.03,0.005,1.030301,0.001697,0.001955,-0.009162,0.000007
4,0.003925,0.012552,2.870,0.133,0.159,0.03,0.005,1.040604,0.001756,0.001959,-0.004561,-0.000032
...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.004001,0.013256,5.668,0.239,0.280,0.03,0.005,2.599273,0.001758,0.002009,-0.020701,-0.000007
97,0.003901,0.012973,5.718,0.229,0.279,0.03,0.005,2.625266,0.001732,0.001982,0.003488,0.000048
98,0.003928,0.012827,5.763,0.223,0.268,0.03,0.005,2.651518,0.001761,0.001949,0.017451,0.000003
99,0.003979,0.013098,5.792,0.234,0.263,0.03,0.005,2.678033,0.001753,0.001892,-0.004992,-0.000007


In [29]:
x1 = scipy.optimize.shgo(estimator,bounds=((0.01,1),(0.01,500),(0.01,1),(0.01,1)), 
                         n=1000, sampling_method='sobol', iters=5)

D:\conda\lib\site-packages\scipy\optimize\_optimize.py:284: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



In [30]:
x1

     fun: 3.7416068380171076e-13
    funl: array([3.74160684e-13, 5.17495631e-13, 8.66175870e-13, 1.25473363e-11,
       2.94138653e-11, 3.21710544e-11, 8.28474186e-11, 1.52860872e-10,
       2.61748737e-10, 2.89946649e-10, 3.57187656e-10, 4.13148282e-10,
       5.01267001e-10, 6.40104555e-10, 6.68409489e-10, 6.69166177e-10,
       8.01281629e-10, 1.07311658e-09, 1.11719573e-09, 1.28704196e-09,
       1.42674710e-09, 1.42699734e-09, 1.59871225e-09, 2.28151433e-09,
       3.01050782e-09, 3.13337945e-09, 3.21119308e-09, 3.25065542e-09,
       3.40035009e-09, 3.49257026e-09, 3.57911805e-09, 4.18408916e-09,
       4.22401544e-09, 4.31197048e-09, 4.64835381e-09, 4.73248370e-09,
       5.67875878e-09, 5.77765882e-09, 6.27470549e-09, 6.73710444e-09,
       6.87781893e-09, 6.92143206e-09, 7.22991704e-09, 7.45105384e-09,
       7.53730087e-09, 7.62928640e-09, 7.70143888e-09, 8.10764792e-09,
       8.39062225e-09, 8.63275104e-09, 8.64577801e-09, 8.80221172e-09,
       8.82083940e-09, 8.98935169e

In [31]:
x1.x

array([  0.50382734, 199.40593353,   0.50565363,   0.49420444])

In [32]:
alpha,1/lambd,c,sigma

(0.5, 200.0, 0.5, 0.5)

In [39]:
def estimator2(x):
    
    a,l,c,s = x
    global scenario
    
    #scenario1=scenario[10:].copy()
    scenario1=scenario[2:].copy()
    
    P = 1-np.exp(-l*scenario1["FC_max"])
    scenario1["g1"] = (1+scenario1["r"])/scenario1["r"]*(scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"].shift()+scenario1["n_en"])))**2-1/l)-scenario1["E"]
    scenario1["g2"] = (1+scenario1["r"])/scenario1["r"]*scenario1["S"]/8*((1-c)/(1+s*(scenario1["n"])))**2-(1/scenario1["r"]-a/(1+scenario1["r"]-a))*1/l-(1+scenario1["r"])/(1+scenario1["r"]-a)*scenario1["FC_max"]
    scenario1["g3"] = scenario1["n"]-scenario1["n"].shift()+(1-a)*(1-P)*scenario1["n"].shift()-P*scenario1["n_en"]
    scenario1["g4"] = -scenario1["FC_max"]*np.exp(-l*scenario1["FC_max"])-np.exp(-l*scenario1["FC_max"])/l+1/l-P*scenario1["FC_mean"]
    
    scenario["g1"]=scenario1["g1"]
    scenario["g2"]=scenario1["g2"]
    scenario["g3"]=scenario1["g3"]
    scenario["g4"]=scenario1["g4"]
    
    return scenario1["g1"].mean()**2+scenario1["g2"].mean()**2+scenario1["g3"].mean()**2+scenario1["g4"].mean()**2
    #return scenario["g1"].mean()**2,scenario["g2"].mean()**2,scenario["g3"].mean()**2,scenario["g4"].mean()**2

In [52]:
tests = pd.DataFrame()
tests["alpha"]=[]
tests["lambda"]=[]
tests["MC"]=[]
tests["sigma"]=[]

for i in range(10):
    sigma = np.random.uniform()
    c = np.random.uniform()
    n = 10
    S = 100



    alpha = np.random.uniform()
    lambd=np.random.uniform(0.001,0.01)#это средние фиксде косты входящего
    CF = 0.001


    epoch = 100
    #r = np.random.normal(loc=0.06, scale=0.01, size=epoch)
    

    data = pd.DataFrame()
    #data["r"] = np.random.normal(0.03,0.01,epoch+1)
    #data["E"] = np.random.normal(0.005,0.001,epoch+1)
    data["r"] = [0.03]*epoch
    data["E"] = [0.005]*epoch


    
    scenario=storyteller(alpha,data["r"],sigma,c,n,S,epoch,lambd,data["E"],CF)
    
    x1 = scipy.optimize.shgo(estimator2,bounds=((0.01,1),(100,1000),(0.01,1),(0.01,1)), 
                         n=1000, sampling_method='sobol', iters=10)
    
    tests.loc[len(tests)] = [f"true alpha: {round(alpha,2)}, estimated alpha: {round(x1.x[0],2)}",
             f"true lambda: {round(1/lambd,2)}, estimated lambda: {round(x1.x[1],2)}",
             f"true MC: {round(c,2)}, estimated MC: {round(x1.x[2],2)}",
             f"true sigma: {round(sigma,2)}, estimated sigma: {round(x1.x[3],2)}"]

AttributeError: 'int' object has no attribute 'iloc'

In [50]:
tests

,alpha,lambda,MC,sigma
0,"true alpha: 0.46, estimated alpha: 0.46","true lambda: 885.25, estimated lambda: 884.42","true MC: 0.41, estimated MC: 0.23","true sigma: 0.69, estimated sigma: 0.9"
1,"true alpha: 0.84, estimated alpha: 0.82","true lambda: 106.86, estimated lambda: 108.79","true MC: 0.85, estimated MC: 0.87","true sigma: 0.91, estimated sigma: 0.82"
2,"true alpha: 0.45, estimated alpha: 0.44","true lambda: 125.58, estimated lambda: 125.81","true MC: 0.74, estimated MC: 0.75","true sigma: 0.82, estimated sigma: 0.76"
3,"true alpha: 0.78, estimated alpha: 0.78","true lambda: 138.41, estimated lambda: 138.43","true MC: 0.82, estimated MC: 0.84","true sigma: 0.14, estimated sigma: 0.12"


In [46]:
data

,r,E,S
0,0.03,0.005,10.000000
1,0.03,0.005,10.300000
2,0.03,0.005,10.609000
3,0.03,0.005,10.927270
4,0.03,0.005,11.255088
5,0.03,0.005,11.592741
6,0.03,0.005,11.940523
7,0.03,0.005,12.298739
8,0.03,0.005,12.667701
9,0.03,0.005,13.047732


In [ ]:
tests = pd.DataFrame()
tests["alpha"]=[]
tests["lambda"]=[]
tests["MC"]=[]
tests["sigma"]=[]

for i in range(100):
    sigma = 0.6
    c = 0.3
    n = 10
    S = 10



    alpha = 0.1
    lambd=0.01#это средние фиксде косты входящего
    CF = 0.001


    epoch = 50
    #r = np.random.normal(loc=0.06, scale=0.01, size=epoch)
    

    data = pd.DataFrame()
    #data["r"] = np.random.normal(0.03,0.01,epoch+1)
    #data["E"] = np.random.normal(0.005,0.001,epoch+1)
    data["r"] = [0.03]*epoch
    data["E"] = [0.005]*epoch
    S=[S]
    gr=1.03
    for i in range(epoch-1):
        S.append(S[-1]*gr)
    data["S"] = S

    
    scenario=storyteller(alpha,data["r"],sigma,c,n,data["S"],epoch,lambd,data["E"],CF)
    
    x1 = scipy.optimize.shgo(estimator2,bounds=((0.01,1),(10,150),(0.01,1),(0.01,1)), 
                         n=500, sampling_method='sobol', iters=1)
    
    tests.loc[len(tests)] = [round(x1.x[0],2),
             round(x1.x[1],2),
             round(x1.x[2],2),
             round(x1.x[3],2)]

In [54]:
tests

,alpha,lambda,MC,sigma
0,0.09,100.11,0.36,0.56
1,0.11,99.90,0.28,0.63
2,0.10,99.91,0.30,0.61
3,0.10,99.97,0.34,0.57
4,0.10,100.27,0.33,0.59
...,...,...,...,...
95,0.10,100.00,0.31,0.60
96,0.11,99.86,0.28,0.63
97,0.10,100.14,0.30,0.61
98,0.10,99.94,0.30,0.61


In [55]:
tests.mean()

alpha       0.1007
lambda    100.0011
MC          0.3154
sigma       0.5949
dtype: float64

In [101]:
tests.std()

alpha     0.008729
lambda    0.163756
MC        0.108469
sigma     0.097619
dtype: float64

In [56]:
alpha,1/lambd,c,sigma

(0.1, 100.0, 0.3, 0.6)